In [1]:

import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm

from lvmsurveysim.utils.sqlite2astropy import peewee2astropy
from lvmdrp.core.image import Image, loadImage, combineImages
from lvmdrp.functions.imageMethod import preprocRawFrame_drp, basicCalibration_drp, createMasterFrame_drp

from lvmdrp.core.constants import CONFIG_PATH
from lvmdrp.main import load_master_config
from lvmdrp.utils.database import ReductionStatus, QualityFlag, create_or_connect_db, LVMFrames, get_raws_metadata


config = load_master_config()
db = create_or_connect_db(config)


In [10]:
lvm_frames = peewee2astropy(LVMFrames).to_pandas().set_index("id")
import pandas as pd

lvm_frames.datetime = lvm_frames.datetime.apply(pd.to_datetime)
lvm_frames.obstime = lvm_frames.obstime.apply(pd.to_datetime)
lvm_frames

,datetime,mjd,spec,ccd,exptime,imagetyp,obstime,observat,label,path,...,ffs,mi150,ts,ldls,nirled,reduction_started,reduction_finished,status,flags,calib_id
id,,,,,,,,,,,,,,,,,,,,,
1,2023-02-13 16:56:44.928418,59786,sp3,z3,5.0,object,2022-07-26 03:54:05.955,SBS,sdR-s-z3-00005247,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
2,2023-02-13 16:56:44.928513,59786,sp3,r3,6.0,object,2022-07-26 03:32:28.081,SBS,sdR-s-r3-00005245,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
3,2023-02-13 16:56:44.928586,59786,sp3,r3,0.0,bias,2022-07-26 03:39:38.822,SBS,sdR-s-r3-00005246,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
4,2023-02-13 16:56:44.928656,59786,sp3,z3,6.0,object,2022-07-26 03:32:28.081,SBS,sdR-s-z3-00005245,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
5,2023-02-13 16:56:44.928724,59786,sp3,r3,5.0,object,2022-07-26 03:54:05.955,SBS,sdR-s-r3-00005247,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19724,2023-02-13 16:56:49.886670,59678,sp1,b1,300.0,continuum,2022-04-09 01:04:27.451,LCO,sdR-s-b1-00003273,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,1,0,1,0,None,None,1,1,None
19725,2023-02-13 16:56:49.886738,59678,sp1,r1,120.0,continuum,2022-04-09 01:27:26.891,LCO,sdR-s-r1-00003274,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,1,0,1,0,None,None,1,1,None
19726,2023-02-13 16:56:49.886837,59678,sp1,z1,2.0,continuum,2022-04-09 00:53:19.274,LCO,sdR-s-z1-00003272,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,1,0,1,0,None,None,1,1,None


In [18]:
from datetime import datetime, timedelta

recent_frames = lvm_frames.loc[lvm_frames.obstime >= datetime.now() - timedelta(days=120)]
recent_cal_frames = recent_frames.loc[lvm_frames.imagetyp.isin(["bias", "dark", "flat"])]

gr = recent_cal_frames.groupby(["ccd", "imagetyp", "exptime"])
gr.groups

{('b1', 'bias', 0.0): [9629], ('b2', 'dark', 5.0): [12703, 12711, 12718, 12727, 12728], ('b3', 'dark', 1.0): [1577, 1596, 1772, 1841], ('b3', 'dark', 3.0): [6183, 6268, 6287, 6295, 6326, 6347, 6351], ('b3', 'dark', 5.0): [2202, 2232, 2326, 2392, 2399, 2404, 2474, 2509, 2517, 2539, 2544, 2548, 2558, 2573, 2576, 15769, 15792, 15850, 16047, 16054, 16065, 16067, 16259, 16401, 16413, 16440, 16452, 16483, 16593, 16677], ('b3', 'dark', 9.0): [6178, 6229, 6243, 6281, 6286, 6291, 6297], ('b3', 'dark', 10.0): [2230, 2238, 2284, 2301, 2341, 2346, 2376, 2422, 2423, 2433, 2437, 2459, 2477, 2566, 2581, 3642, 3644, 3681, 3705, 3716, 3762, 3769, 3992, 4027, 4158, 4166, 4250, 15758, 15835, 15996, 16014, 16061, 16072, 16108, 16146, 16207, 16431, 16442, 16458, 16488, 16522, 16678], ('b3', 'dark', 20.0): [6214, 6232, 6241, 6261, 6308, 6325, 6341], ('b3', 'dark', 30.0): [1564, 1585, 1605, 1722, 1735, 1771, 1809, 1826, 1903, 1914, 2194, 2203, 2209, 2226, 2231, 2275, 2281, 2305, 2374, 2387, 2458, 2473, 2545,